<a href="https://colab.research.google.com/github/ahyz0569/Final-project-practice/blob/colab/200324_doc2vec_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Doc2vec 사용해보기

colab에서 doc2vec 모델을 학습시키고 문장 유사도를 확인해보자

In [0]:
# mecab 설치
# 출처: https://somjang.tistory.com/entry/Google-Colab에서-Mecab-koMecab-ko-dic-쉽게-사용하기 [솜씨좋은장씨]

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
! bash install_mecab-ko_on_colab190912.sh

In [0]:
!pwd

/content/Mecab-ko-for-Google-Colab


## Library import

In [0]:
import gensim
import sys
import pandas as pd
from konlpy.tag import Mecab
m = Mecab()
import re
import os

# imports package
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import RegexpTokenizer

## 데이터 로드 및 전처리
- 특수 문자 제거

In [0]:
df=pd.read_csv('/content/drive/My Drive/Final-project/NLP/data/10000recipe_dataframe_for_token.csv',
                sep='\t',
                encoding='utf-8')
df.head()

,id,title,steps
0,6894825,"#냉장고를부탁해 #이연복 전복영양밥만들기 #전복과 오징어,새우가 들어간 볶음밥에 모...",먼저 전복을 물에 넣고는 삶아 줍니다. 전복을 삶은 중간에 오징어와 새우를 넣어서 ...
1,6894824,목우촌 캠핑요리 베이컨 야채말이,준비해둔 야채를 깨끗이 씻고 깻잎은 절반 대파는 한입크기 파프리카는 채 썰어줍니다 ...
2,6894823,단호박라떼 & 전자렌지로 단호박 찌는법,단호박 찌는법 첫번째는 단호박은 반을 자르고 속안에 든 씨를 모두 제거해 줍니다. ...
3,6894822,"풀무원 ""보글보글 된장찌게"" 끓이기",풀무원 정통된장을 냄비나 뚝배기에 물을 500ml넣고 끓여주세요.\\n(뒷면에 자세...
4,6894820,미숫가루 만들기,모든잡곡은 고두밥을 지어요 율무쌀은 가장 단단한 잡곡 이예요 불리지않고 밥하는것처럼...


In [0]:
# 문자열 내 특수 문자 제거 함수 선언 
def cleanText(readData):
    #텍스트에 포함되어 있는 특수 문자 제거 
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', str(readData)) 
    return text

In [0]:
# title 컬럼 특수문자 제거하기
df['title']=df['title'].apply(cleanText)

In [0]:
df.head()

,id,title,steps
0,6894825,냉장고를부탁해 이연복 전복영양밥만들기 전복과 오징어새우가 들어간 볶음밥에 모닝글로리...,먼저 전복을 물에 넣고는 삶아 줍니다. 전복을 삶은 중간에 오징어와 새우를 넣어서 ...
1,6894824,목우촌 캠핑요리 베이컨 야채말이,준비해둔 야채를 깨끗이 씻고 깻잎은 절반 대파는 한입크기 파프리카는 채 썰어줍니다 ...
2,6894823,단호박라떼 전자렌지로 단호박 찌는법,단호박 찌는법 첫번째는 단호박은 반을 자르고 속안에 든 씨를 모두 제거해 줍니다. ...
3,6894822,풀무원 보글보글 된장찌게 끓이기,풀무원 정통된장을 냄비나 뚝배기에 물을 500ml넣고 끓여주세요.\\n(뒷면에 자세...
4,6894820,미숫가루 만들기,모든잡곡은 고두밥을 지어요 율무쌀은 가장 단단한 잡곡 이예요 불리지않고 밥하는것처럼...


In [0]:
# 특수문자 제거 후 새로 저장
df.to_csv('/content/drive/My Drive/Final-project/NLP/data/10000recipe_dataframe_for_token.csv',
            sep='\t',
            encoding='utf-8',
            index=False)

## 레시피 제목으로 doc2vec 학습시키기

In [0]:
# 데이터 프레임 불러오기
df=pd.read_csv('/content/drive/My Drive/Final-project/NLP/data/10000recipe_dataframe_for_token.csv',
                sep='\t',
                encoding='utf-8')
df.head()

,id,title,steps
0,6894825,냉장고를부탁해 이연복 전복영양밥만들기 전복과 오징어새우가 들어간 볶음밥에 모닝글로리...,먼저 전복을 물에 넣고는 삶아 줍니다. 전복을 삶은 중간에 오징어와 새우를 넣어서 ...
1,6894824,목우촌 캠핑요리 베이컨 야채말이,준비해둔 야채를 깨끗이 씻고 깻잎은 절반 대파는 한입크기 파프리카는 채 썰어줍니다 ...
2,6894823,단호박라떼 전자렌지로 단호박 찌는법,단호박 찌는법 첫번째는 단호박은 반을 자르고 속안에 든 씨를 모두 제거해 줍니다. ...
3,6894822,풀무원 보글보글 된장찌게 끓이기,풀무원 정통된장을 냄비나 뚝배기에 물을 500ml넣고 끓여주세요.\\n(뒷면에 자세...
4,6894820,미숫가루 만들기,모든잡곡은 고두밥을 지어요 율무쌀은 가장 단단한 잡곡 이예요 불리지않고 밥하는것처럼...


In [0]:
df.shape

(54738, 3)

In [0]:
# 레시피 제목과 id만 추려서 데이터프레임 새로 만들기
title_df = df[['id', 'title']]
title_df.head()

,id,title
0,6894825,냉장고를부탁해 이연복 전복영양밥만들기 전복과 오징어새우가 들어간 볶음밥에 모닝글로리...
1,6894824,목우촌 캠핑요리 베이컨 야채말이
2,6894823,단호박라떼 전자렌지로 단호박 찌는법
3,6894822,풀무원 보글보글 된장찌게 끓이기
4,6894820,미숫가루 만들기


In [0]:
# 문장을 토큰화 시킬 함수 선언
def tokenizer_Mecab_morphs(doc):
    return m.morphs(str(doc))

def tokenizer_Mecab_noun(doc):
    return m.nouns(str(doc))

def tokenizer_Mecab_pos(doc):
    return m.pos(str(doc), flatten=True)

In [0]:
# 함수 사용 테스트
print(tokenizer_Mecab_morphs('더 좋은 한 끼 곤약젤리'))
print(tokenizer_Mecab_noun('더 좋은 한 끼 곤약젤리'))
print(tokenizer_Mecab_pos('더 좋은 한 끼 곤약젤리'))

['더', '좋', '은', '한', '끼', '곤약', '젤리']
['끼', '곤약', '젤리']
[('더', 'MAG'), ('좋', 'VA'), ('은', 'ETM'), ('한', 'MM'), ('끼', 'NNBC'), ('곤약', 'NNG'), ('젤리', 'NNG')]


In [0]:
# title 컬럼을 토큰화시켜서 새로운 컬럼('token_title') 생성
title_df['token_title']=title_df['title'].apply(tokenizer_Mecab_morphs)

In [0]:
title_df.head()

,id,title,token_title
0,6894825,냉장고를부탁해 이연복 전복영양밥만들기 전복과 오징어새우가 들어간 볶음밥에 모닝글로리...,"[냉장고, 를, 부탁, 해, 이연복, 전복, 영양, 밥, 만들, 기, 전복, 과, ..."
1,6894824,목우촌 캠핑요리 베이컨 야채말이,"[목, 우촌, 캠핑, 요리, 베이컨, 야채, 말, 이]"
2,6894823,단호박라떼 전자렌지로 단호박 찌는법,"[단호박, 라떼, 전자, 렌지, 로, 단호박, 찌, 는, 법]"
3,6894822,풀무원 보글보글 된장찌게 끓이기,"[풀무원, 보글보글, 된장, 찌, 게, 끓이, 기]"
4,6894820,미숫가루 만들기,"[미숫가루, 만들, 기]"


## Doc2vec 모델 학습시키기

In [0]:
# Doc2vec 입력 형식에 맞게 데이터 수정
# tags = 문서 ID
# words = 단어 list
title_doc_df = title_df[['id', 'token_title']].values.tolist()
tagged_data = [TaggedDocument(words=_d, tags=[uid]) for uid, _d in title_doc_df]

In [0]:
# Training
max_epochs=10

model=Doc2Vec(window=10,
             vector_size=150,
             alpha=0.025,
             min_alpha=0.025,
             min_count=2,
             dm = 1,
             negative = 5,
             seed=9999)

model.build_vocab(tagged_data)
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
               total_examples=model.corpus_count,
               epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    model.min_alpha = model.alpha

iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [0]:
# 모델 저장하기
from gensim.test.utils import get_tmpfile

temp_path = get_tmpfile('title_doc2vec_0325.pkl')
DOC2VEC_DRIVE_PATH = '/content/drive/My Drive/Final-project/NLP/model/'
model.save(os.path.join(DOC2VEC_DRIVE_PATH, "title_doc2vec_0325.pkl"))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# temp 파일 삭제
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

## 문장 유사도 확인하기

In [0]:
# model load
DOC2VEC_MODEL_PATH = os.path.join(DOC2VEC_DRIVE_PATH, "title_doc2vec_0325.pkl")
model = Doc2Vec.load(DOC2VEC_MODEL_PATH)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# 새로운 문장을 넣어 유사도 확인하기
model.random.seed(8000)

example_list = '참치 김치찌개'.split(' ')

inferred_vector = model.infer_vector(example_list)
return_docs = model.docvecs.most_similar(positive=[inferred_vector],topn=20)
for rd in return_docs:
  for des in title_df[title_df['id'] == rd[0]]['title']:
    print(rd[0], rd[1], des)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


6928007 0.7900514006614685 100 통밀쿠키 만들기
6918472 0.7760583758354187 ♥신혼밥상 홈메이드 칼로리바
6911922 0.7713984847068787 가족모임요리 우렁된장찌개
6757123 0.7628169059753418 두릅 데치기
6879682 0.7613463997840881 얼그레이 파운드 케이크
6870471 0.7543428540229797 도다리쑥국 맛있는 도다리쑥국 끓이는 법
6871605 0.7488991618156433 호떡믹스로 공갈빵 만들기
6878198 0.7476012110710144 노생크림 까르보나라 만들기
6915758 0.7470545768737793 노버터 생크림 파운드케익
6885690 0.7466774582862854 콩나물밥 슥슥 비벼먹는 한그릇음식
6916208 0.7443692088127136 ♥신혼밥상 닭가슴살 유린기
6857927 0.7441118955612183 폭탄계란찜 달걀찜 만들기
6918027 0.7425355911254883 암예방레시피 묵은지콩비지찌개
6897323 0.7418239712715149 서리태 콩자반
6834621 0.741104781627655 토란국 끓이는 법
6847864 0.7376580238342285 백선생표 프렌치 토스트
6925432 0.7364944815635681 소세지꼬치  소떡소떡 만들기
6927076 0.7355220913887024 크래미 계란말이
6853025 0.7350984811782837 호박식혜만들기 단호박식혜
4441836 0.7344920039176941 오징어먹물크림파스타


In [0]:
# 단어로 유사도 확인해보기
model.most_similar('김치',topn=20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('파김치', 0.5638792514801025),
 ('열무김치', 0.5317940711975098),
 ('겉절이', 0.5102351307868958),
 ('깍두기', 0.49384286999702454),
 ('무생채', 0.48973560333251953),
 ('어묵', 0.47553691267967224),
 ('김치찌개', 0.467621386051178),
 ('갓김치', 0.46203505992889404),
 ('김장', 0.4584785997867584),
 ('카레', 0.4547005295753479),
 ('두부김치', 0.45323556661605835),
 ('김치볶음밥', 0.4419439733028412),
 ('배추김치', 0.44108328223228455),
 ('콩나물', 0.4394105076789856),
 ('백김치', 0.4384535551071167),
 ('열무', 0.4379288852214813),
 ('배', 0.4246397018432617),
 ('오이소박이', 0.42393097281455994),
 ('라면', 0.41968291997909546),
 ('오징어', 0.41711878776550293)]

## 결과

- 단어를 넣어 유사도를 확인한 경우에는 유사 단어가 잘 나오지만 문장을 넣었을 경우에는 원하는 결과 값이 잘 나오지 않는 것을 확인하였음
- 문장을 토큰화 한 후 불용어 제거를 하지 않고 모델에 학습시켰더니 결과가 좋지 않은 것으로 보인다
- 불용어 제거를 한 후 모델의 성능을 확인 해봐야 겠다.